In [1]:
import pandas as pd
from textblob import TextBlob
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [2]:
songlist=pd.read_csv('../data/processed_data.csv')
print(songlist.columns)
songlist.drop(columns=['Unnamed: 0','Unnamed: 0.1'],inplace=True)
songlist.head()

Index(['Unnamed: 0.1', 'Unnamed: 0', 'artist_name', 'track_uri', 'artist_uri',
       'track_name', 'album_uri', 'duration', 'album_name', 'artists',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'artist_pop', 'genres', 'track_pop'],
      dtype='object')


,artist_name,track_uri,artist_uri,track_name,album_uri,duration,album_name,artists,danceability,energy,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_pop,genres,track_pop
0,Kadie Elder,72ey4Cr2eXXwH9IMlhzuCV,spotify:artist:6y2lKfMfaxfcEmHwU058FG,First Time He Kissed a Boy,spotify:album:1GYbbLzBosZi7MLG12iasq,187977,First Time He Kissed a Boy,Kadie Elder,0.616,0.856,...,audio_features,72ey4Cr2eXXwH9IMlhzuCV,spotify:track:72ey4Cr2eXXwH9IMlhzuCV,https://api.spotify.com/v1/tracks/72ey4Cr2eXXw...,https://api.spotify.com/v1/audio-analysis/72ey...,187978,4,26,unknown,44
1,Matt Alber,3vsHwtwcRXjwejyfKgj6dX,spotify:artist:3v2p34eAq6fINTEyJ1w2vK,Handsome Man,spotify:album:5ysULa9YY0ccEAZtxklzln,240920,Wind Sand Stars,Matt Alber,0.319,0.176,...,audio_features,3vsHwtwcRXjwejyfKgj6dX,spotify:track:3vsHwtwcRXjwejyfKgj6dX,https://api.spotify.com/v1/tracks/3vsHwtwcRXjw...,https://api.spotify.com/v1/audio-analysis/3vsH...,240920,4,23,unknown,20
2,Steve Grand,7i2xzOJU1O7hrP8kqqBchZ,spotify:artist:0oqk73d6vKVx8Ekz4onThW,All-American Boy,spotify:album:40nTEfg3tTj1bptrG7IQU0,297410,All American Boy,Steve Grand,0.441,0.709,...,audio_features,7i2xzOJU1O7hrP8kqqBchZ,spotify:track:7i2xzOJU1O7hrP8kqqBchZ,https://api.spotify.com/v1/tracks/7i2xzOJU1O7h...,https://api.spotify.com/v1/audio-analysis/7i2x...,297410,4,25,queer_country,34
3,Adam Lambert,44aN5xKL3kGHvQ5bXVk6B8,spotify:artist:6prmLEyn4LfHlD9NnXWlf7,Ghost Town,spotify:album:3kaQUt8Mp906u1fI0LDqO6,208330,The Original High (Deluxe Version),Adam Lambert,0.703,0.736,...,audio_features,44aN5xKL3kGHvQ5bXVk6B8,spotify:track:44aN5xKL3kGHvQ5bXVk6B8,https://api.spotify.com/v1/tracks/44aN5xKL3kGH...,https://api.spotify.com/v1/audio-analysis/44aN...,208330,4,61,dance_pop idol pop post-teen_pop,67
4,Adam Lambert,44aN5xKL3kGHvQ5bXVk6B8,spotify:artist:6prmLEyn4LfHlD9NnXWlf7,Ghost Town,spotify:album:3kaQUt8Mp906u1fI0LDqO6,208330,The Original High (Deluxe Version),Adam Lambert,0.703,0.736,...,audio_features,44aN5xKL3kGHvQ5bXVk6B8,spotify:track:44aN5xKL3kGHvQ5bXVk6B8,https://api.spotify.com/v1/tracks/44aN5xKL3kGH...,https://api.spotify.com/v1/audio-analysis/44aN...,208330,4,61,dance_pop idol pop post-teen_pop,67


In [3]:
songlist[['artist_name','track_name','track_uri']]

,artist_name,track_name,track_uri
0,Kadie Elder,First Time He Kissed a Boy,72ey4Cr2eXXwH9IMlhzuCV
1,Matt Alber,Handsome Man,3vsHwtwcRXjwejyfKgj6dX
2,Steve Grand,All-American Boy,7i2xzOJU1O7hrP8kqqBchZ
3,Adam Lambert,Ghost Town,44aN5xKL3kGHvQ5bXVk6B8
4,Adam Lambert,Ghost Town,44aN5xKL3kGHvQ5bXVk6B8
...,...,...,...
578,k.d. lang,Constant Craving,5TK55y7SXDt36Fi14WBG8X
579,Ariane Moffatt,Walls of the World,2nTu3jikKUP01bD5GkxgoD
580,Courtney Barnett,Pedestrian at Best,7uoQRNx100em2FLLxnqym0
581,Mary Lambert,Secrets,3V3iy4K6paycRmTyrjQVwi


In [4]:
def drop_duplicate(df):
    '''
    Drop duplicate songs
    '''
    df['artists_song'] = df.apply(lambda row: row['artist_name']+row['track_name'],axis = 1)
    return df.drop_duplicates('artists_song')
songlist=drop_duplicate(songlist)
print("Are all songs unique: ",len(pd.unique(songlist.artists_song))==len(songlist))

Are all songs unique:  True


In [5]:
def select_columns(df):
    return df[['artist_name','id','track_name','danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', "artist_pop", "genres", "track_pop"]]
songlist=select_columns(songlist)
songlist.head()

,artist_name,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,genres,track_pop
0,Kadie Elder,72ey4Cr2eXXwH9IMlhzuCV,First Time He Kissed a Boy,0.616,0.856,11,-4.901,0,0.0389,0.0238,0.000000,0.5980,0.630,107.013,26,unknown,44
1,Matt Alber,3vsHwtwcRXjwejyfKgj6dX,Handsome Man,0.319,0.176,7,-11.104,1,0.0393,0.8630,0.000000,0.1060,0.306,130.310,23,unknown,20
2,Steve Grand,7i2xzOJU1O7hrP8kqqBchZ,All-American Boy,0.441,0.709,6,-3.910,1,0.0333,0.0840,0.000000,0.0851,0.280,131.889,25,queer_country,34
3,Adam Lambert,44aN5xKL3kGHvQ5bXVk6B8,Ghost Town,0.703,0.736,4,-5.709,0,0.0460,0.1860,0.000019,0.1740,0.274,119.955,61,dance_pop idol pop post-teen_pop,67
5,Queen,09jZZLarDD6MI3eT3ujsEL,I Want To Break Free,0.862,0.402,4,-12.075,1,0.0368,0.0478,0.004990,0.1790,0.812,109.157,83,classic_rock glam_rock rock,53


In [6]:
def genre_preprocess(df):
    '''
    Preprocess the genre data
    '''
    df['genres_list'] = df['genres'].apply(lambda x: x.split(" "))
    return df
songlist= genre_preprocess(songlist)
songlist['genres_list'].head()

0                                [unknown]
1                                [unknown]
2                          [queer_country]
3    [dance_pop, idol, pop, post-teen_pop]
5          [classic_rock, glam_rock, rock]
Name: genres_list, dtype: object

In [7]:
def playlist_preprocesses(df):
    df=drop_duplicate(df)
    df=select_columns(df)
    df=genre_preprocess(df)
    return df

Now the recommender system is actually coded

In [8]:
def getSubjectivity(text):
  '''
  Getting the Subjectivity using TextBlob
  '''
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  '''
  Getting the Polarity using TextBlob
  '''
  return TextBlob(text).sentiment.polarity

def getAnalysis(score, task="polarity"):
  '''
  Categorizing the Polarity & Subjectivity score
  '''
  if task == "subjectivity":
    if score < 1/3:
      return "low"
    elif score > 1/3:
      return "high"
    else:
      return "medium"
  else:
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

def sentiment_analysis(df, text_col):
  '''
  Perform sentiment analysis on text
  ---
  Input:
  df (pandas dataframe): Dataframe of interest
  text_col (str): column of interest
  '''
  df['subjectivity'] = df[text_col].apply(getSubjectivity).apply(lambda x: getAnalysis(x,"subjectivity"))
  df['polarity'] = df[text_col].apply(getPolarity).apply(getAnalysis)
  return df

In [9]:
sentiment = sentiment_analysis(songlist, "track_name")
sentiment.head()

,artist_name,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,genres,track_pop,genres_list,subjectivity,polarity
0,Kadie Elder,72ey4Cr2eXXwH9IMlhzuCV,First Time He Kissed a Boy,0.616,0.856,11,-4.901,0,0.0389,0.0238,0.000000,0.5980,0.630,107.013,26,unknown,44,[unknown],medium,Positive
1,Matt Alber,3vsHwtwcRXjwejyfKgj6dX,Handsome Man,0.319,0.176,7,-11.104,1,0.0393,0.8630,0.000000,0.1060,0.306,130.310,23,unknown,20,[unknown],high,Positive
2,Steve Grand,7i2xzOJU1O7hrP8kqqBchZ,All-American Boy,0.441,0.709,6,-3.910,1,0.0333,0.0840,0.000000,0.0851,0.280,131.889,25,queer_country,34,[queer_country],low,Neutral
3,Adam Lambert,44aN5xKL3kGHvQ5bXVk6B8,Ghost Town,0.703,0.736,4,-5.709,0,0.0460,0.1860,0.000019,0.1740,0.274,119.955,61,dance_pop idol pop post-teen_pop,67,"[dance_pop, idol, pop, post-teen_pop]",low,Neutral
5,Queen,09jZZLarDD6MI3eT3ujsEL,I Want To Break Free,0.862,0.402,4,-12.075,1,0.0368,0.0478,0.004990,0.1790,0.812,109.157,83,classic_rock glam_rock rock,53,"[classic_rock, glam_rock, rock]",high,Positive


In [10]:
def ohe_prep(df, column, new_name): 
    ''' 
    Create One Hot Encoded features of a specific column
    ---
    Input: 
    df (pandas dataframe): Spotify Dataframe
    column (str): Column to be processed
    new_name (str): new column name to be used
        
    Output: 
    tf_df: One-hot encoded features 
    '''
    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [16]:
subject_ohe = ohe_prep(sentiment, 'subjectivity','subject')
subject_ohe.iloc[0]

subject|high      0
subject|low       0
subject|medium    1
Name: 0, dtype: uint8

In [20]:
tfidf = TfidfVectorizer()
tfidf_matrix =  tfidf.fit_transform(songlist['genres_list'].apply(lambda x: " ".join(x)))
genre_df = pd.DataFrame(tfidf_matrix.toarray())
genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
genre_df.drop(columns='genre|unknown')
genre_df.reset_index(drop = True, inplace=True)
genre_df.iloc[0]

genre|_brasileira     0.0
genre|_hip_hop        0.0
genre|aarhus_indie    0.0
genre|acoustic_pop    0.0
genre|afrofuturism    0.0
                     ... 
genre|viral_pop       0.0
genre|viral_rap       0.0
genre|vocal_house     0.0
genre|vogue           0.0
genre|women           0.0
Name: 0, Length: 288, dtype: float64

In [21]:
print(songlist['artist_pop'].describe())

count    554.000000
mean      37.384477
std       23.694145
min        0.000000
25%       17.000000
50%       38.000000
75%       56.000000
max       88.000000
Name: artist_pop, dtype: float64


In [22]:
pop = songlist[["artist_pop"]].reset_index(drop = True)
scaler = MinMaxScaler()
pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns)
pop_scaled.head()

,artist_pop
0,0.295455
1,0.261364
2,0.284091
3,0.693182
4,0.943182


In [23]:
def create_feature_set(df, float_cols):
    '''
    Process spotify df to create a final set of features that will be used to generate recommendations
    ---
    Input: 
    df (pandas dataframe): Spotify Dataframe
    float_cols (list(str)): List of float columns that will be scaled
            
    Output: 
    final (pandas dataframe): Final set of features 
    '''
    
    # Tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['genres_list'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
    genre_df.drop(columns='genre|unknown') # drop unknown genre
    genre_df.reset_index(drop = True, inplace=True)
    
    # Sentiment analysis
    df = sentiment_analysis(df, "track_name")

    # One-hot Encoding
    subject_ohe = ohe_prep(df, 'subjectivity','subject') * 0.3
    polar_ohe = ohe_prep(df, 'polarity','polar') * 0.5
    key_ohe = ohe_prep(df, 'key','key') * 0.5
    mode_ohe = ohe_prep(df, 'mode','mode') * 0.5

    # Normalization
    # Scale popularity columns
    pop = df[["artist_pop","track_pop"]].reset_index(drop = True)
    scaler = MinMaxScaler()
    pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns) * 0.2 

    # Scale audio columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    # Concanenate all features
    final = pd.concat([genre_df, floats_scaled, pop_scaled, subject_ohe, polar_ohe, key_ohe, mode_ohe], axis = 1)
    
    # Add song id
    final['id']=df['id'].values
    
    return final

In [25]:
float_cols = songlist.dtypes[songlist.dtypes == 'float64'].index.values
songlist.to_csv("../data/allsong_data.csv", index = False)

# Generate features
complete_feature_set = create_feature_set(songlist, float_cols=float_cols)
complete_feature_set.to_csv("../data/complete_feature.csv", index = False)
complete_feature_set.head()

/home/arghya/anaconda3/envs/queerify/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,genre|_brasileira,genre|_hip_hop,genre|aarhus_indie,genre|acoustic_pop,genre|afrofuturism,genre|alabama_rap,genre|alt_z,genre|alternative_country,genre|alternative_dance,genre|alternative_pop,...,key|5,key|6,key|7,key|8,key|9,key|10,key|11,mode|0,mode|1,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,72ey4Cr2eXXwH9IMlhzuCV
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.5,3vsHwtwcRXjwejyfKgj6dX
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,7i2xzOJU1O7hrP8kqqBchZ
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,44aN5xKL3kGHvQ5bXVk6B8
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,09jZZLarDD6MI3eT3ujsEL


In [34]:
playlistDF_test=pd.read_csv('../data/test_playlist.csv')
playlistDF_test=playlist_preprocesses(playlistDF_test)
playlistDF_test.head()
playlistDF_test.to_csv("../data/test_playlist.csv")

In [30]:
def generate_playlist_feature(complete_feature_set, playlist_df):
    '''
    Summarize a user's playlist into a single vector
    ---
    Input: 
    complete_feature_set (pandas dataframe): Dataframe which includes all of the features for the spotify songs
    playlist_df (pandas dataframe): playlist dataframe
        
    Output: 
    complete_feature_set_playlist_final (pandas series): single vector feature that summarizes the playlist
    complete_feature_set_nonplaylist (pandas dataframe): 
    '''
    
    # Find song features in the playlist
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]
    # Find all non-playlist song features
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]
    complete_feature_set_playlist_final = complete_feature_set_playlist.drop(columns = "id")
    return complete_feature_set_playlist_final.sum(axis = 0), complete_feature_set_nonplaylist

In [35]:
complete_feature_set_playlist_vector, complete_feature_set_nonplaylist = generate_playlist_feature(complete_feature_set, playlistDF_test)

In [36]:
complete_feature_set_nonplaylist.head()

,genre|_brasileira,genre|_hip_hop,genre|aarhus_indie,genre|acoustic_pop,genre|afrofuturism,genre|alabama_rap,genre|alt_z,genre|alternative_country,genre|alternative_dance,genre|alternative_pop,...,key|5,key|6,key|7,key|8,key|9,key|10,key|11,mode|0,mode|1,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,72ey4Cr2eXXwH9IMlhzuCV
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.5,3vsHwtwcRXjwejyfKgj6dX
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,7i2xzOJU1O7hrP8kqqBchZ
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,44aN5xKL3kGHvQ5bXVk6B8
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,09jZZLarDD6MI3eT3ujsEL


In [37]:
# Summarized playlist features
complete_feature_set_playlist_vector

genre|_brasileira     0.0
genre|_hip_hop        0.0
genre|aarhus_indie    0.0
genre|acoustic_pop    0.0
genre|afrofuturism    0.0
                     ... 
key|9                 0.0
key|10                0.0
key|11                0.0
mode|0                0.0
mode|1                0.0
Length: 319, dtype: float64

In [38]:
def generate_playlist_recos(df, features, nonplaylist_features):
    '''
    Generated recommendation based on songs in aspecific playlist.
    ---
    Input: 
    df (pandas dataframe): spotify dataframe
    features (pandas series): summarized playlist feature (single vector)
    nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Output: 
    non_playlist_df_top_40: Top 40 recommendations for that playlist
    '''
    
    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    # Find cosine similarity between the playlist and the complete song set
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    
    return non_playlist_df_top_40

In [40]:
# Genreate top 10 recommendation
recommend = generate_playlist_recos(songlist, complete_feature_set_playlist_vector, complete_feature_set_nonplaylist)
recommend.head(10)

,artist_name,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,artist_pop,genres,track_pop,genres_list,subjectivity,polarity,sim
0,Kadie Elder,72ey4Cr2eXXwH9IMlhzuCV,First Time He Kissed a Boy,0.616,0.856,11,-4.901,0,0.0389,0.0238,...,0.5980,0.6300,107.013,26,unknown,44,[unknown],medium,Positive,0.0
396,Bobby Guard,0XUHRNm4yHhC9XcfURMFL7,When All The World,0.545,0.398,5,-10.490,1,0.0363,0.1660,...,0.0963,0.0765,110.020,5,unknown,0,[unknown],low,Neutral,0.0
390,Jake Wesley Rogers,5DPwVhgevj57PZ3LlBMeVs,The Pretender,0.546,0.217,2,-13.558,1,0.0314,0.7190,...,0.2450,0.3460,108.129,42,modern_alternative_pop,32,[modern_alternative_pop],low,Neutral,0.0
391,Pikes,4TqEoafKpX51TnO21qHMzg,Thylacine,0.562,0.122,7,-18.338,1,0.0330,0.4460,...,0.1630,0.0789,107.142,24,unknown,0,[unknown],low,Neutral,0.0
392,Zoe Konez,0OYQ24ngz67JFs4GNWA3KL,My One Love,0.439,0.148,11,-16.610,0,0.0649,0.7680,...,0.2280,0.1310,171.735,19,unknown,24,[unknown],high,Positive,0.0
393,Kristóf Hajós,4Ee40EHrdpT2agDeuVYYNx,Love is Cancelled,0.746,0.510,6,-9.378,1,0.0288,0.5660,...,0.1650,0.6210,125.046,5,unknown,1,[unknown],high,Positive,0.0
394,boy cam,4A74rEokmBC1nXf1wtMmqi,I'm Happy Now,0.577,0.430,4,-12.564,1,0.0355,0.5260,...,0.1100,0.1470,124.482,19,unknown,22,[unknown],high,Positive,0.0
395,Myylo,5YSzJkh3hbuab1I8OBYjbS,I Do,0.492,0.594,4,-6.422,1,0.0305,0.1180,...,0.2020,0.5550,84.808,32,unknown,7,[unknown],low,Neutral,0.0
397,Ben Platt,6AhoLcPx9fkz8BRqCZtZTL,River,0.624,0.131,7,-9.588,1,0.0314,0.9470,...,0.1070,0.3510,101.038,64,hollywood pop show_tunes,29,"[hollywood, pop, show_tunes]",low,Neutral,0.0
405,a balladeer,2MfyphgyjIrH8jIvktfVxc,Hail Mary,0.387,0.234,10,-11.960,1,0.0384,0.7910,...,0.1210,0.3620,149.737,40,dutch_indie dutch_pop dutch_rock,14,"[dutch_indie, dutch_pop, dutch_rock]",low,Neutral,0.0


In [41]:
playlistDF_test[["artist_name","track_name"]][:20]

,artist_name,track_name
0,Harry Styles,As It Was
1,Joji,Glimpse of Us
2,Charlie Puth,Left and Right (Feat. Jung Kook of BTS)
3,Kate Bush,Running Up That Hill (A Deal With God)
4,Bad Bunny,Me Porto Bonito
5,Bad Bunny,Tití Me Preguntó
6,Bad Bunny,Ojitos Lindos
7,Bad Bunny,Efecto
8,Beyoncé,BREAK MY SOUL
9,Bad Bunny,Moscow Mule
